In [1]:
!pip install datasets
!pip install transformers[torch]

  Using cached numpy-2.0.0-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
  Using cached huggingface_hub-0.23.4-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/547.8 kB ? eta -:--:--
   ---- ----------------------------------- 61.4/547.8 kB 3.2 MB/s eta 0:00:01
   ------------------ --------------------- 256.0/547.8 kB 3.2 MB/s eta 0:00:01
   ---------------------------------------  542.7/547.8 kB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 547.8/547.8 kB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
   ---------------------------------------- 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/316.1 kB ? eta -:--:--
   --------------------------------------- 316.1/316.1 kB 19.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/370.8 kB ? eta -:--:--
   -------------------------------

In [4]:
!pip install scikit-learn

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.8 kB 435.7 kB/s eta 0:00:01
     -------------------------------- ----- 51.2/60.8 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 459.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.0 MB 1.7 MB/s eta 0:00:07
    --------------------------------------- 0.2/11.0 MB 2.2 MB/s eta 0:00:05
   - -------------------------------------- 0.5/11.0 MB 3.6 MB/s eta 0:00:03
   -- ------------------------------------- 0.8/11.0 MB 4.1 MB/s eta 0:00:03
   ----- ---------------------------------- 1.5/11.0 MB 6.2 MB/s eta 0:00:02
   ---------- ----------------------------- 2.9/11.0 MB 10.2 MB/s eta 0:00:01
   ------------------- -------------------- 5.4/11.0 MB 16.5 MB/s eta 0:00:01
   -

In [4]:
!pip install --upgrade transformers
!pip install --upgrade huggingface_hub

     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     ------------------------------------- -- 41.0/43.6 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 43.6/43.6 kB 710.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
    --------------------------------------- 0.1/9.3 MB 2.4 MB/s eta 0:00:04
   - -------------------------------------- 0.4/9.3 MB 4.2 MB/s eta 0:00:03
   --- ------------------------------------ 0.7/9.3 MB 5.0 MB/s eta 0:00:02
   ----- ---------------------------------- 1.2/9.3 MB 6.5 MB/s eta 0:00:02
   ---------- ----------------------------- 2.4/9.3 MB 10.2 MB/s eta 0:00:01
   -------------------- ------------------- 4.7/9.3 MB 17.5 MB/s eta 0:00:01
   --------------------------- ------------ 6.3/9.3 MB 21.3 MB/s eta 0:00:01
   ---------------------------- ----------- 6.8/9.3 MB 19.6 MB/s eta 0:00:01
   ------------------------------ --------- 7.1/9.3 MB 18.0 MB/s eta 0:00:01
   -------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2023.4.0 requires fsspec==2023.4.0, but you have fsspec 2024.6.1 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_metric

d:\Anaconda3\envs\transfomers\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# 데이터 로드
dataset = load_dataset('csv', data_files={'train': 'train_test2.csv', 'test':'test_test2.csv'})

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

# 텍스트 데이터 토크나이즈
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 라벨 설정 및 불필요한 컬럼 제거
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets.set_format("torch")

# 모델 로드
model = AutoModelForSequenceClassification.from_pretrained('klue/bert-base', num_labels=43)

# 훈련 설정
training_args = TrainingArguments(
    output_dir='result/result2_model_4',
    evaluation_strategy='epoch',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
)

# 평가 지표 정의
metric = load_metric('accuracy', trust_remote_code=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# 트레이너 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)

# 모델 훈련
trainer.train()

Map: 100%|██████████| 649/649 [00:00<00:00, 1055.29 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\Anaconda3\envs\transfomers\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


KeyboardInterrupt: 